# LSH Example

With the IMDB data and minhash signatures, use LSH to find similar pairs

In [1]:
%matplotlib inline

In [2]:
import json
import random
import pandas as pd

from scipy.sparse import lil_matrix

from sklearn.metrics import jaccard_score

In [3]:
actor_id_to_name_map = {}     # Map Actor IDs to actor names
actor_id_to_index_map = {}    # Map actor IDs to a unique index of known actors
index_to_actor_ids = []       # Array mapping unique index back to actor ID (invert of actor_id_to_index_map)

index_counter = 0    # Unique actor index; increment for each new actor
known_actors = set()

movie_actor_list = [] # List of all our movies and their actors

test_count = 0
with open("imdb_recent_movies.json", "r") as in_file:
    for line in in_file:
        
        this_movie = json.loads(line)
            
        # Keep track of all the actors in this movie
        for actor_id,actor_name in zip(this_movie['actor_ids'],this_movie['actor_names']):
            
            # Keep names and IDs
            actor_id_to_name_map[actor_id] = actor_name
            
            # If we've seen this actor before, skip...
            if actor_id in known_actors:
                continue
                
            # ... Otherwise, add to known actor set and create new index for them
            known_actors.add(actor_id)
            actor_id_to_index_map[actor_id] = index_counter
            index_to_actor_ids.append(actor_id)
            index_counter += 1
            
        # Finished with this film
        movie_actor_list.append({
            "movie": this_movie["title_name"],
            "actors": set(this_movie['actor_ids'])
        })


In [4]:
print("Known Actors:", len(known_actors))

Known Actors: 258059


## Convert to MinHash Reduced Dimension

For min-hashing, we first generate some number of permutations that will drive our "min hash" of the first non-zero element in the permuted column set (i.e., after shuffling columns, find the first permuted column idex with a non-zero element).

Note that, here, we _never_ store the full matrix of Movies X Actors.

In [5]:
permutations = []

# How many hashing functions to use
#  What happens as you change this number?
#  Larger values make things slower but provide better Jaccard estimates
num_minhashes = 128

# Generate permutations for hashing d-dimensional features into k minhash dimensions
for i in range(num_minhashes):
    column_indices = list(range(len(known_actors)))
    
    # Shuffle the original indices
    random.shuffle(column_indices)
    
    # Generate a map of the actual index to its permuted index
    permutations.append({j:i for i,j in enumerate(column_indices)})

In [7]:
minhash_rows = []

test_count = 0
with open("imdb_recent_movies.json", "r") as in_file:
    for line in in_file:
        
        this_movie = json.loads(line)
            
        this_movie_actor_indices = []
        for actor_id in this_movie['actor_ids']:
            this_movie_actor_indices.append(actor_id_to_index_map[actor_id])
            
        minhash_sig = []
        for permutation in permutations:
            
            # Find the first index of actors in this movie using the permuted indices map
            permed_index = sorted([permutation[i] for i in this_movie_actor_indices])[0]
            minhash_sig.append(permed_index)
        
        minhash_rows.append(minhash_sig)
        


In [8]:
minhash_df = pd.DataFrame(
    minhash_rows,
    index=[m["movie"] for m in movie_actor_list],
)
minhash_df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
The Other Side of the Wind,64662,52018,73,28789,65000,86428,77369,131100,25737,53277,...,136646,78932,4807,149735,150844,145217,114323,205657,92458,21974
November 1828,5704,69373,38896,35569,33469,29037,4404,23235,8794,65384,...,67465,137821,30964,38822,1718,9236,22143,14870,5575,14511
The Drive to Win,52869,1922,161139,11982,49120,179874,58446,4958,13504,72780,...,34168,76607,144324,22625,62743,80249,16458,56824,5180,142135
The Naked Monster,129074,50239,23929,108771,221977,204127,72358,40386,102454,12102,...,69637,154298,71413,98006,87476,30004,29511,60392,19945,172488
El día de los albañiles 2,239088,41,22674,69661,15499,21629,184042,173877,73378,147265,...,51081,43020,67878,135125,86404,99006,52248,132388,102661,102642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Secret of China,118924,9456,17148,69912,66125,112280,32370,67237,21234,33153,...,5066,127722,98407,11011,29606,5268,24455,127420,9803,43330
Kuambil Lagi Hatiku,73745,191895,111597,82813,223932,134982,217621,22131,54076,167876,...,108981,18119,17367,15412,223365,153578,6443,21998,104752,240393
Rodolpho Teóphilo - O Legado de um Pioneiro,147759,193945,154759,118850,40312,141158,25473,243619,252967,151487,...,109294,224925,6816,224651,110577,252696,253826,52599,152813,172512
Dankyavar Danka,80820,76241,180210,10218,90086,101118,120261,67208,36783,40757,...,17106,18674,165310,94259,43429,1736,29890,121773,90464,128225


In [9]:
query_row = minhash_df.loc["Star Wars: Episode II - Attack of the Clones"]
minh_similarities = []

for idx,row in minhash_df.iterrows():
    score = sum(query_row == row) / len(row)
    minh_similarities.append((idx, score))
    

In [10]:
most_sim_mh = sorted(minh_similarities, key=lambda d: d[1], reverse=True)[:20]
most_sim_mh

[('Star Wars: Episode II - Attack of the Clones', 1.0),
 ('Star Wars: Episode III - Revenge of the Sith', 0.4765625),
 ('Whales of Atlantis: In Search of Moby Dick', 0.3671875),
 ('Necessary Evil: Super-Villains of DC Comics', 0.3671875),
 ('Angels in Notting Hill', 0.28125),
 ('Little Italy', 0.28125),
 ('Faster', 0.2734375),
 ('Virgin Territory', 0.2734375),
 ("Troy's Story", 0.2734375),
 ('Miss Potter', 0.2734375),
 ('The Final Fix', 0.2734375),
 ('Perfect Sense', 0.2734375),
 ('Fastest', 0.2734375),
 ('Charge', 0.2734375),
 ('Ice Bear', 0.2734375),
 ('Hugo', 0.2578125),
 ('Awake', 0.25),
 ('Life as a House', 0.2421875),
 ('Young Adam', 0.234375),
 ('Down with Love', 0.234375)]

In [73]:
lsh_buckets = {}

rows = 4
bands = int(minhash_df.shape[1] / rows)
print("Bands:", bands, "Rows:", rows)

Bands: 32 Rows: 4


In [74]:
counter = 0
for idx,row in minhash_df.iterrows():
    
    for b in range(bands):
        segment = ",".join(["%d" % d for d in row[b*rows:(b+1)*rows]])
        bucket = hash(segment) % 2**20
        
        collision_set = lsh_buckets.get(bucket, set())
        collision_set.add(idx)
        
        lsh_buckets[bucket] = collision_set
    
    counter += 1
    
    if counter % 1000 == 0:
        print(time.asctime(), counter)

Tue Feb 22 15:27:56 2022 1000
Tue Feb 22 15:27:57 2022 2000
Tue Feb 22 15:27:58 2022 3000
Tue Feb 22 15:27:59 2022 4000
Tue Feb 22 15:28:00 2022 5000
Tue Feb 22 15:28:01 2022 6000
Tue Feb 22 15:28:02 2022 7000
Tue Feb 22 15:28:03 2022 8000
Tue Feb 22 15:28:04 2022 9000
Tue Feb 22 15:28:05 2022 10000
Tue Feb 22 15:28:08 2022 11000
Tue Feb 22 15:28:09 2022 12000
Tue Feb 22 15:28:10 2022 13000
Tue Feb 22 15:28:11 2022 14000
Tue Feb 22 15:28:12 2022 15000
Tue Feb 22 15:28:13 2022 16000
Tue Feb 22 15:28:14 2022 17000
Tue Feb 22 15:28:15 2022 18000
Tue Feb 22 15:28:16 2022 19000
Tue Feb 22 15:28:17 2022 20000
Tue Feb 22 15:28:18 2022 21000
Tue Feb 22 15:28:19 2022 22000
Tue Feb 22 15:28:20 2022 23000
Tue Feb 22 15:28:21 2022 24000
Tue Feb 22 15:28:22 2022 25000
Tue Feb 22 15:28:23 2022 26000
Tue Feb 22 15:28:24 2022 27000
Tue Feb 22 15:28:25 2022 28000
Tue Feb 22 15:28:26 2022 29000
Tue Feb 22 15:28:28 2022 30000
Tue Feb 22 15:28:29 2022 31000
Tue Feb 22 15:28:30 2022 32000
Tue Feb 22 15:28:

In [75]:
target_movie = "The Matrix Reloaded"

query_row = minhash_df.loc[target_movie]

candidates = set()
for bucket in lsh_buckets.values():
    if target_movie in bucket:
        candidates = candidates.union(bucket)

In [76]:
len(candidates)

152

In [79]:
for c in candidates:

    right_q = minhash_df.loc[c]
    
    score = 0.0
    if len(right_q.shape) > 1:
        for subidx,subrow in right_q.iterrows():
            score = sum(query_row == subrow) / len(subrow)
            if score > 0:
                print(c)
                print("\t", score)
    else:
        score = sum(query_row == right_q) / len(right_q)
        if score > 0:
            print(c)
            print("\t", score)


The Matrix Revolutions
	 1.0
John Wick: Chapter 3 - Parabellum
	 0.5078125
The Matrix Reloaded
	 1.0


In [80]:
with open("lsh_serial.json", "w") as out_file:
    json.dump([list(bucket) for bucket in lsh_buckets.values()], out_file)